<a href="https://colab.research.google.com/github/victori4sa/Lab-DEL/blob/main/Sistema%20de%20gerenciamento%20de%20despesas%20pessoais%20(comentado%20e%20c/%20salvamento%20apenas%20no%20in%C3%ADcio%20e%20no%20fim).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Sistema de gerenciamento de despesas pessoais

import os          # Usado para verificar a existência de arquivos (ex.: os.path.exists)
import hashlib     # Usado para gerar hashes, como a senha criptografada dos usuários
import datetime    # Usado para registrar datas e horários (ex.: nas despesas)


# Função que retorna as configurações
def obter_configuracoes():
    return "usuarios.txt" # Retorna o nome do arquivo onde os usuários serão armazenados


# Função que gera o nome do arquivo de despesas do usuário
def despesas_filename(usuario):
    return "%s_despesas.txt" % usuario # Monta o nome do arquivo usando o nome do usuário


# Inicializa o arquivo de usuários caso ele ainda não exista
def inicializar_usuarios(users_file):
    # Verifica se o arquivo informado não existe
    if not os.path.exists(users_file):
        # Abre o arquivo no modo escrita ("w"), criando um novo arquivo vazio se ele não existir
        # O encoding="utf-8" garante que caracteres com acento funcionem
        f = open(users_file, "w", encoding="utf-8")
        try: # Se acontecer qualquer erro, o python pula pro finally
            pass
        finally: # Sempre fecha o arquivo, mesmo se der errado no try
            f.close()


# Converte a senha para um hash (código), para que no arquivo de senhas fiquem só os hashes, não senhas reais
def hash_senha(senha):
    # Transforma a senha em bytes (encode) e gera o hash em formato hexadecimal
    return hashlib.sha256(senha.encode("utf-8")).hexdigest() # SHA-256 é o algoritmo responsável por essa transformação


# Carrega todos os usuários em um dicionário
def carregar_usuarios(users_file):
    # Dicionário onde cada chave será o nome do usuário e o valor um outro dicionário com os dados dele
    usuarios = {}

    # Se o arquivo não existir, retorna um dicionário vazio
    if not os.path.exists(users_file):
        return usuarios

    # Abre o arquivo para leitura
    f = open(users_file, "r", encoding="utf-8")
    try:
        for linha in f: # Lê cada linha do arquivo
            linha = linha.strip() # Remove quebras de linha e espaços extras

            if linha == "": # Ignora linhas vazias
                continue

            partes = linha.split(";") # Cada dado é separado por ponto e vírgula

            # Se a linha não tiver pelo menos nome e senha, ignora
            if len(partes) < 2:
                continue

            # Pega cada informação da linha, mesmo se algum campo estiver faltando
            nome = partes[0]
            senha_hash = partes[1] if len(partes) > 1 else ""
            salario = partes[2].strip() if len(partes) > 2 and partes[2].strip() != "" else ""
            estado = partes[3].strip() if len(partes) > 3 and partes[3].strip() != "" else ""
            # "estaado" guarda onde o usuário estava antes de sair (a última vez)

            # Armazena os dados desse usuário no dicionário principal
            usuarios[nome] = {"senha_hash": senha_hash, "salario": salario, "estado": estado}
    finally: # Fecha o arquivo mesmo se ocorrer algum erro
        f.close()

    return usuarios # Retorna o dicionário com todos os usuários carregados


# Salva todos os usuários no arquivo
def salvar_usuarios(usuarios, users_file):
    # Abre o arquivo no modo escrita, sobrescrevendo o conteúdo antigo
    f = open(users_file, "w", encoding="utf-8")
    try:
        # Para cada usuário no dicionário, monta uma linha com seus dados
        for nome, info in usuarios.items():
            linha = "%s;%s;%s;%s\n" % (nome, info.get("senha_hash", ""), info.get("salario", ""), info.get("estado", ""))
            # Escreve a linha no arquivo
            f.write(linha)
    finally:
        f.close() # Garante que o arquivo será fechado mesmo se ocorrer erro


# Função que cadastra um novo usuário usando o dicionário já carregado
def cadastrar(usuarios, users_file):
    # Lê o nome do novo usuário
    nome = input("Nome de usuário: ").strip()

    # Verifica se o nome está vazio
    if nome == "":
        print("Nome inválido.")
        return

    # Verifica se o nome já existe
    if nome in usuarios:
        print("Usuário já existe.")
        return

    # Verifica se contém caracteres inválidos (ex.: ponto e vírgula)
    if ";" in nome or " " in nome:
        print("Nome inválido. Não pode conter ';' ou espaços.")
        return

    # Lê a senha e verifica se não está vazia
    senha = input("Senha: ").strip()
    if senha == "":
        print("Senha inválida.")
        return

    # Pergunta o salariop
    salario_input = input("Informe seu salário mensal (ou deixe em branco para definir depois): R$ ").strip()

    if salario_input == "": # Se o salário for vazio, grava como vazio
        salario_para_gravar = ""
    else:
        # Tenta converter para número; se falhar, cancela o cadastro
        try:
            salario_val = float(salario_input)
            if salario_val <= 0:  # Verifica se o salário é menor ou igual à zero.
                print("Salário inválido. Cadastro cancelado.")
                return
            salario_para_gravar = "%.2f" % salario_val
        except Exception:
            print("Valor de salário inválido. Cadastro cancelado.")
            return

    # Salva os dados do novo usuário no dicionário
    usuarios[nome] = {
        "senha_hash": hash_senha(senha), # Guarda o hash da senha
        "salario": salario_para_gravar, # Guarda o salário já formatado
        "estado": "" # Estado começa vazio
    }

    salvar_usuarios(usuarios, users_file) # Atualiza o arquivo de usuários

    # Cria o arquivo de despesas desse usuário
    f = open(despesas_filename(nome), "w", encoding="utf-8")
    try:
        pass
    finally:
        f.close()

    print("Usuário %s cadastrado com sucesso." % nome)


# Função que realiza o login usando o dicionário de usuários já carregado
def fazer_login(usuarios):
    # Lê o nome do usuário e a senha digitados
    nome = input("Usuário: ").strip()
    senha = input("Senha: ").strip()

    # Verifica se o usuário existe no dicionário
    if nome not in usuarios:
        print("Usuário não encontrado.") # Imprime isso caso não esteja.
        return "", "", []

    info = usuarios[nome]

    # Compara o hash da senha digitada com o hash salvo
    if info.get("senha_hash", "") != hash_senha(senha):
        print("Senha incorreta.")
        return "", "", []

    print("Login realizado com sucesso!")

    # Carrega as despesas desse usuário uma única vez
    lista_despesas = carregar_despesas(nome)

    return nome, info, lista_despesas


# Atualiza um campo específico (ex: salário, estado) de um usuário
def atualizar_campo_usuario(usuarios, nome, campo, valor):
    # Se o usuário não existir, não faz nada
    if nome not in usuarios:
        return
    # Atualiza o campo desejado com o novo valor
    usuarios[nome][campo] = valor


# Carrega todas as despesas de um usuário
def carregar_despesas(usuario):
    arquivo = despesas_filename(usuario)
    despesas = []

    # Se o arquivo não existir, retorna a lista vazia
    if not os.path.exists(arquivo):
        return despesas

    # Abre o arquivo de despesas do usuário
    f = open(arquivo, "r", encoding="utf-8")
    try:
        # Lê cada linha do arquivo
        for linha in f:
            linha = linha.strip()

            if linha == "": # Ignora as linhas vazias
                continue

            partes = linha.split(";") # Cada despesa é separada por ";"
            if len(partes) < 4:
                continue

            data = partes[0]
            categoria = partes[1]

            # Converte o valor para número; se falhar, usa 0.0
            try:
                valor = float(partes[2])
            except Exception:
                valor = 0.0

            descricao = partes[3]

            # Adiciona a despesa à lista
            despesas.append({"data": data, "categoria": categoria, "valor": valor, "descricao": descricao})
    finally:
        f.close()

    return despesas


# Lê uma data digitada pelo usuário e vê se o formato e válido
def ler_data_comparacao(prompt):
    while True:
        entrada_usuario = input(prompt).strip() # .strip: remove espaços extras no início/fim para evitar erro no strptime
        try: # Tenta converter a data no formato DD-MM-YYYY
            return datetime.datetime.strptime(entrada_usuario, "%d-%m-%Y").date()
        except Exception: # Se der erro, avisa e pede novamente
            print("Data inválida. Use o formato DD-MM-YYYY.")

# Filtra as despesas que estão dentro de um intervalo de datas
def filtrar_periodo_comparacao(lista_despesas, data_inicio, data_fim):
    resultado = []

    # Percorre todas as despesas
    for d in lista_despesas:
        try: # Converte a data salva no formato "HH:MM:SS_DD-MM-YYYY" para objeto de data
            data_obj = datetime.datetime.strptime(d["data"], "%H:%M:%S_%d-%m-%Y").date()
        except Exception: # Se a data estiver inválida, pula para a próxima despesa
            continue

        # Verifica se a data está dentro do período informado
        if data_inicio <= data_obj <= data_fim:
            resultado.append(d)

    return resultado


# Mostra o menu inicial e retorna a opção escolhida
def menu_inicial():
    while True:
        print("Opções:")
        print("1 - Cadastrar usuário")
        print("2 - Fazer login")
        print("3 - Sair")

        escolha = input("O que deseja? ").strip() #Remove os esspaços extras no innício e no fim

        # Se for uma opção válida, retorna
        if escolha in ("1", "2", "3"):
            return escolha
        else:
            print("Opção inválida. Digite 1, 2 ou 3.")


# Exibe o menu principal do usuário e permite retomar o último estado salvo
def menu_principal(usuario, info_usuario, usuarios, lista_despesas):

    # Se existir um estado salvo, o programa informa onde o usuário parou e pergunta se ele deseja continuar exatamente daquela opção.
    # Caso o usuário responda "s", a variável 'cmd' é definida com o estado salvo, para que retome de onde parou.
    # Caso o usuário responda "n", 'cmd' é esvaziada e o menu recomeça normalmente.

    estado_salvo = info_usuario.get("estado", "") or ""

    if estado_salvo != "":
        print("Você saiu da última vez enquanto estava na opção: %s" % estado_salvo)
        resp = input("Deseja continuar dessa opção? (s/n): ").strip().lower()
        if resp == "s":
            cmd = estado_salvo
        else:
            cmd = ""
    else:
        cmd = ""

    # Loop que mostra as opções do menu e executa o que o usuário escoher.
    while True:
        print("Opções:")
        print("1 - Adicionar despesa")
        print("2 - Listar despesas")
        print("3 - Total geral")
        print("4 - Total por categoria")
        print("5 - Saldo restante do mês")
        print("6 - Gerar relatório")
        print("7 - Selecionar período e comparar")
        print("8 - Alterar salário")
        print("9 - Sair")

        # Se cmd estiver vazio, o usuário deve escolher a opção agora.
        if cmd == "":
            cmd = input("O que deseja? ").strip()

        info_usuario["estado"] = cmd # Salva a opção atual no estado do usuário, para poder continuar de onde parou.

        if cmd == "1": # Adicionar despesa
            categoria = input("Categoria: ").strip()
            if categoria == "":
                print("Categoria inválida.")
            else:
                descricao = input("Descrição (opcional): ").strip()
                valor_str = input("Valor (ex: 12.50): R$ ").strip()

                # Tenta converter o valor digitado.
                try:
                    valor = float(valor_str.replace(",", ".")) # para que aceite inserir tanto nº c/ "," quanto c/ ".".

                    ts = datetime.datetime.now().strftime("%H:%M:%S_%d-%m-%Y") # Gera data/hora atual para registrar a despesa.

                    # Salva a despesa na lista em memória.
                    lista_despesas.append({"data": ts, "categoria": categoria, "valor": valor, "descricao": descricao})
                    print("Despesa adicionada: %s | %s | R$ %.2f" % (ts, categoria, valor))
                except Exception:
                    print("Valor inválido. Despesa não registrada.")

        elif cmd == "2": # Listar despesas
            if not lista_despesas:
                print("Nenhuma despesa registrada.")
            else:
                print("Lista de despesas do usuário %s:" % usuario)
                for d in lista_despesas:
                    print("%s | %-15s | R$ %7.2f | %s" % (d["data"], d["categoria"], d["valor"], d["descricao"]))
                    # %-15s -> alinha a categoria à esquerda, ocupando 15 espaços, para ficar alinhado.
                    # R$ %7.2f -> mostra o valor em reais com 2 casas decimais, ocupando 7 espaços para alinhar os números.

        elif cmd == "3": # Total geral

            # Soma todos os valores registrados até agora.
            total = 0
            for d in lista_despesas:
                total += d["valor"]
            print("Total geral: R$ %7.2f" % total)

        elif cmd == "4": # Total por categoria

            # Agrupa despesas por categoria e soma os valores.
            totais = {}
            for d in lista_despesas:
                cat = d["categoria"]
                val = d["valor"]
                if cat not in totais:
                    totais[cat] = 0.0
                totais[cat] += val

            if not totais:
                print("Nenhuma despesa registrada.")
            else:
                for cat, val in totais.items():
                    print("%s: R$ %7.2f" % (cat, val))

        elif cmd == "5": # Saldo restante do mÊs

            # Verifica se o salário já existe.
            salario_salvo = info_usuario.get("salario", "")
            if salario_salvo.strip() == "":
                while True:
                    # Se não existir, pede o salário até o usuário digitar um valor válido.
                    entrada_salario = input("Salário mensal não encontrado. Digite seu salário (ex: 2500.00): R$ ").strip()
                    try:
                        salario_val = float(entrada_salario)
                        salario_para_gravar = "%.2f" % salario_val
                        info_usuario["salario"] = salario_para_gravar
                        print("Salário salvo.")
                        break
                    except Exception:
                        print("Valor inválido. Digite um número válido.")

            try: # Converte o salário armazenado para número.
                salario_float = float(info_usuario.get("salario", "0") or 0.0)
            except Exception:
                salario_float = 0.0

            # Soma todas as despesas.
            total = 0.0
            for d in lista_despesas:
                total += d["valor"]

            saldo = salario_float - total

            print("Salário: R$ %7.2f" % salario_float)
            print("Saldo restante: R$ %7.2f" % saldo)


        elif cmd == "6": # Gerar relatório

            # Garante que o salário foi informado antes de gerar o relatório.
            salario_salvo = info_usuario.get("salario", "")
            if salario_salvo.strip() == "":
                while True:
                    entrada_salario_relatorio = input("Digite seu salário para calcular o saldo (ex: 2500.00): R$ ").strip()
                    try:
                        salario_val = float(entrada_salario_relatorio)
                        salario_salvo = "%.2f" % salario_val
                        info_usuario["salario"] = salario_salvo
                        print("Salário salvo.")
                        break
                    except Exception:
                        print("Valor inválido. Tente novamente.")

            # Converte o salário.
            try:
                salario_float = float(salario_salvo)
            except Exception:
                salario_float = 0.0

            # Calcula total por categoria.
            totais_cat = {}
            for d in lista_despesas:
                cat = d["categoria"]
                val = d["valor"]
                if cat not in totais_cat:
                    totais_cat[cat] = 0.0
                totais_cat[cat] += val

            # Total geral.
            total = 0.0
            for d in lista_despesas:
                total += d["valor"]

            saldo = salario_float - total

            # Gera o nome do arquivo com data e horário.
            agora = datetime.datetime.now().strftime("%H-%M-%S_%d-%m-%Y")
            nome_arquivo = "relatorio_%s_%s.txt" % (usuario, agora)

            # Abre o arquivo e escreve todo o conteúdo.
            f = open(nome_arquivo, "w", encoding="utf-8")
            f.write("RELATÓRIO DE DESPESAS - Usuário: %s\n" % usuario)
            f.write("Gerado em: %s\n\n" % (agora.replace("_", " ")))

            f.write("Lista de despesas:\n")
            if not lista_despesas:
                f.write("(nenhuma despesa)\n")
            else:
                for d in lista_despesas:
                    f.write("%s | %s | R$ %.2f | %s\n" % (d["data"], d["categoria"], d["valor"], d["descricao"]))

            f.write("\nTotais por categoria:\n")
            if not totais_cat:
                f.write("(nenhuma categoria)\n")
            else:
                for cat, val in totais_cat.items():
                    f.write("%s: R$ %.2f\n" % (cat, val))

            f.write("\nTotal geral: R$ %.2f\n" % total)
            f.write("Salário informado: R$ %.2f\n" % salario_float)
            f.write("Saldo restante: R$ %.2f\n" % saldo)
            f.close()

            print("Relatório gerado: %s" % nome_arquivo)

        elif cmd == "7": # Comparar períodos
            if not lista_despesas:
                print("Nenhuma despesa registrada para comparar.")
            else:
                print("Período 1:")
                p1i = ler_data_comparacao("  Início (DD-MM-YYYY): ")
                p1f = ler_data_comparacao("  Fim   (DD-MM-YYYY): ")

                print("Período 2:")
                p2i = ler_data_comparacao("  Início (DD-MM-YYYY): ")
                p2f = ler_data_comparacao("  Fim   (DD-MM-YYYY): ")

                # Filtra as despesas que estão dentro de cada período.
                periodo1 = filtrar_periodo_comparacao(lista_despesas, p1i, p1f)
                periodo2 = filtrar_periodo_comparacao(lista_despesas, p2i, p2f)

                # Soma os valores dos dois períodos.
                total1 = 0.0
                for d in periodo1:
                    total1 += d["valor"]

                total2 = 0.0
                for d in periodo2:
                    total2 += d["valor"]

                print("Total período 1: R$ %.2f" % total1)
                print("Total período 2: R$ %.2f" % total2)

                # Compara os períodos.
                if total1 > total2:
                    print("Você gastou MAIS no período 1.")
                elif total2 > total1:
                    print("Você gastou MAIS no período 2.")
                else:
                    print("Os períodos têm o mesmo valor gasto.")

        elif cmd == "8": # Alterar salário
            while True:
                novo_salario = input("Digite o novo salário (ex: 2500.00): R$ ").strip()
                try:
                    salario_val = float(novo_salario)
                    salario_formatado = "%.2f" % salario_val
                    info_usuario["salario"] = salario_formatado
                    print("Salário atualizado com sucesso!")
                    break
                except Exception:
                    print("Valor inválido. Tente novamente.")

        elif cmd == "9": # Sair.
            # Salva todas as despesas no arquivo do usuário.
            arquivo = despesas_filename(usuario)
            f = open(arquivo, "w", encoding="utf-8")
            for d in lista_despesas:
                f.write("%s;%s;%.2f;%s\n" % (d["data"], d["categoria"], d["valor"], d["descricao"]))
            f.close()

            # Limpa o estado salvo para não retomar nada na próxima execução.
            usuarios[usuario]["estado"] = ""

            # Salva no arquivo de usuários (salário, estado etc.).
            salvar_usuarios(usuarios, obter_configuracoes())

            print("Saindo...")
            break

        else:
            print("Opção inválida. Digite um número de 1 a 9.")

        # Reseta o comando para que o menu pergunte novamente na próxima repetição do loop.
        cmd = ""


# Função principal
def main():

    # Pega o nome do arquivo de usuários.
    users_file = obter_configuracoes()

    # Cria o arquivo de usuários caso não exista.
    inicializar_usuarios(users_file)

    # Carrega todos os usuários existentes em memória.
    usuarios = carregar_usuarios(users_file)

    while True:
        # Exibe o menu inicial e pega a escolha do usuário
        escolha = menu_inicial()

        if escolha == "1": # Cadastrar ym novo usuário.
            cadastrar(usuarios, users_file)

        elif escolha == "2": # Fazer login
            usuario, info, lista_despesas = fazer_login(usuarios)

            # Se o login for feito, chama o menu principal do usuário logado.
            if usuario != "":
                menu_principal(usuario, info, usuarios, lista_despesas)

                # Grava usuários apenas ao final, se houve alterações
                salvar_usuarios(usuarios, users_file)

                # Pergunta se o usuário tem certeza de que quer encerrar o programa após sair do menu principal
                resp = input("Tem certeza que deseja encerrar o programa? (s/n): ").strip().lower()
                if resp == "s":
                    print("Programa encerrado. Até mais!")
                    break

        elif escolha == "3": # Sair do programa
            print("Encerrando. Até mais!")
            break


# O Python verifica se este arquivo está sendo executado diretamente (não importado como módulo).
if __name__ == "__main__":
    main() # Se sim, chama a função main() que inicia toda a execução do sistema.

Opções:
1 - Cadastrar usuário
2 - Fazer login
3 - Sair
O que deseja? 3
Encerrando. Até mais!
